In [1]:
from tensorflow import keras
model = keras.models.load_model('baseline 2 saved/baseline_2')

2022-06-07 13:54:00.576774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 13:54:00.620907: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 13:54:00.621622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 13:54:00.622898: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## Prepare Tokenizer 

In [2]:
# Tokenize the text to get the vocab count 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

def import_tokenizer(file_path):
    with open(file_path, 'rb') as token:
        tokenizer = pickle.load(token)
    return tokenizer

x_tokenizer = import_tokenizer('baseline 2 saved/x_tokenizer.pickle')
y_tokenizer = import_tokenizer('baseline 2 saved/y_tokenizer.pickle')


In [3]:
#Building Dictionary for Source Vocabulary
reverse_target_word_index=y_tokenizer.index_word 
reverse_source_word_index=x_tokenizer.index_word 
target_word_index=y_tokenizer.word_index
y_vocab = y_tokenizer.num_words + 1

In [14]:
model.input[1]

<KerasTensor: shape=(None, None) dtype=float32 (created by layer 'input_1')>

## Inference

In [15]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, \
    Concatenate, TimeDistributed
from tensorflow.keras.models import Model

latent_dim = 256
embedding_dim = 256
max_len_content = 500
max_len_summary = 62


#Inference/Validation Phase
#Encoding the input sequence to get the feature vector
encoder_inputs = model.input[0]
encoder_outputs, state_h, state_c = model.layers[6].output

encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

#Decoder setup
#These tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_content,latent_dim))

#Getting the embeddings of the decoder sequence
dec_emb2= model.layers[5].output

#Setting the initial states to the states from the previous time step for better prediction
decoder_lstm = model.layers[7]
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#Attention inference
attn_layer = model.layers[8]

attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
#decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

decoder_inf_concat = model.layers[9]([decoder_outputs2, attn_out_inf])

#Adding Dense softmax layer to generate proability distribution over the target vocabulary
decoder_dense = model.layers[10]
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

#Final Decoder model
decoder_inputs = model.input[1]
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [16]:
#Function defining the implementation of inference process
import numpy as np
def decode_sequence(input_seq):
    #Encoding the input as state vectors
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1,1))
    
    #Populating the first word of target sequence with the start word
    target_seq[0, 0] = target_word_index['START']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        #Sampling a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
            continue
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if sampled_token != 'END':
            decoded_sentence += ' '+sampled_token

        #Exit condition: either hit max length or find stop word
        if (sampled_token == 'END' or len(decoded_sentence.split()) >= (max_len_summary-1)):
            stop_condition = True

        #Updating the target sequence (of length 1)
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        #Updating internal states
        e_h, e_c = h, c

    return decoded_sentence

In [17]:
#Functions to convert an integer sequence to a word sequence for summary as well as reviews 
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['START']) and i!=target_word_index['END']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [21]:

from nltk import flatten

def inference(text, padding_type='post', truncation_type='post'):
    text = [text]
    text_infer = x_tokenizer.texts_to_sequences(text)
    text_infer = pad_sequences(text_infer, maxlen=max_len_content, padding=padding_type, truncating=truncation_type)
    text_infer = decode_sequence(text_infer.reshape(1,max_len_content))
    return text_infer
    

In [24]:
def print_prediction(text):
    prediction = inference(text)
    print('Original Text\n')
    print(text)
    print('Predicted Summary\n')
    print(prediction)